**Install dependencies**

In [11]:
!pip install nltk spacy transformers scikit-learn gradio pyngrok
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 120.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


**Import standard libraries**

In [12]:
import pandas as pd
import numpy as np
import re
import json
import random
import joblib


**Import ML libraries**

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


**Import NLP libraries**

In [14]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import spacy


**Import Transformer + Gradio**

In [15]:
from transformers import pipeline
import gradio as gr


**Download NLTK resources**

In [16]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')  # Fixes LookupError


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

**Initialize NLP tools**

In [17]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
nlp = spacy.load("en_core_web_sm")


**Define sample dataset (intents)**

In [18]:
data = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": [
                "Hello",
                "Hi",
                "Hey",
                "Good morning",
                "Good afternoon"
            ],
            "responses": [
                "Hello! How can I assist you today?",
                "Hi there! What can I help you with?",
                "Hey! How can I support you?"
            ]
        },
        {
            "tag": "goodbye",
            "patterns": [
                "Bye",
                "See you later",
                "Goodbye",
                "Have a good day"
            ],
            "responses": [
                "Goodbye! Have a great day!",
                "See you later!",
                "Bye! Don't hesitate to reach out if you need more help."
            ]
        },
        {
            "tag": "thanks",
            "patterns": [
                "Thank you",
                "Thanks",
                "That's helpful",
                "Appreciate it"
            ],
            "responses": [
                "You're welcome!",
                "Happy to help!",
                "Anytime!",
                "Glad I could assist you!"
            ]
        },
        {
            "tag": "password_reset",
            "patterns": [
                "I forgot my password",
                "Can you help me reset my password?",
                "How do I change my password?",
                "Password reset",
                "I need to reset my login"
            ],
            "responses": [
                "You can reset your password by visiting the portal and clicking 'Forgot Password'.",
                "To reset your password, go to the login page and select the password reset option.",
                "I can help with password reset. Please visit https://example.com/password-reset"
            ]
        },
        {
            "tag": "system_access",
            "patterns": [
                "I can't access the system",
                "Login problems",
                "Unable to sign in",
                "System access issue",
                "Authentication problem"
            ],
            "responses": [
                "Please check your username and password. If issues persist, contact IT support at support@example.com.",
                "System access issues are often resolved by clearing your browser cache or trying a different browser.",
                "I've reported this access issue to our IT team. They will contact you shortly."
            ]
        },
        {
            "tag": "software_issue",
            "patterns": [
                "The software is crashing",
                "Application error",
                "Program not working",
                "Bug report",
                "Software problem"
            ],
            "responses": [
                "Please try restarting the application. If the problem persists, try reinstalling.",
                "I've logged your software issue. Our team will investigate this problem.",
                "Could you provide more details about the error message you're seeing?"
            ]
        },
        {
            "tag": "hardware_issue",
            "patterns": [
                "My computer won't turn on",
                "Printer not working",
                "Hardware problem",
                "Device malfunction",
                "Monitor issues"
            ],
            "responses": [
                "For hardware issues, please contact our IT hardware support team at hardware@example.com.",
                "Try checking all connections and cables. If the problem persists, contact hardware support.",
                "I've created a ticket for your hardware issue. A technician will contact you soon."
            ]
        },
        {
            "tag": "network_issue",
            "patterns": [
                "Internet connection problems",
                "Can't connect to network",
                "VPN not working",
                "WiFi issues",
                "Network downtime"
            ],
            "responses": [
                "Try restarting your router or connecting to a different network.",
                "For VPN issues, please ensure you have the latest client installed.",
                "Our network team is aware of this issue and working on a resolution."
            ]
        },
        {
            "tag": "meeting_scheduling",
            "patterns": [
                "Schedule a meeting",
                "Set up a call",
                "Plan a conference",
                "Arrange a meeting",
                "Need to schedule a discussion"
            ],
            "responses": [
                "You can schedule a meeting using our calendar system at https://example.com/calendar",
                "I can help you schedule a meeting. What time were you thinking?",
                "Please use the scheduling tool in Outlook to arrange a meeting."
            ]
        },
        {
            "tag": "email_issue",
            "patterns": [
                "Email not working",
                "Can't send emails",
                "Outlook problems",
                "Email configuration",
                "Inbox issues"
            ],
            "responses": [
                "Try checking your internet connection and restarting your email client.",
                "Email issues are often resolved by updating your password or checking server settings.",
                "I've reported this to our email admin team. They will contact you shortly."
            ]
        }
    ]
}

**Save intents to JSON**

In [19]:
with open('intents.json', 'w') as f:
    json.dump(data, f)


**Preprocessing function**

In [20]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    return ' '.join(tokens)


**Create dataset (patterns → tags)**

In [21]:
tags, patterns = [], []
for intent in data['intents']:
    for pattern in intent['patterns']:
        tags.append(intent['tag'])
        patterns.append(pattern)

df = pd.DataFrame({'text': patterns, 'tag': tags})
df['processed'] = df['text'].apply(preprocess_text)
df.head()


,text,tag,processed
0,Hello,greeting,hello
1,Hi,greeting,hi
2,Hey,greeting,hey
3,Good morning,greeting,good morning
4,Good afternoon,greeting,good afternoon


**Train/test split**

In [22]:
X_train, X_test, y_train, y_test = train_test_split(
    df['processed'], df['tag'], test_size=0.2, random_state=42
)


**Train model**

In [23]:
model = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', SVC(kernel='linear', probability=True))
])
model.fit(X_train, y_train)


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', SVC(kernel='linear', probability=True))])

**Evaluate model**

In [24]:
y_pred = model.predict(X_test)
print(f"Model Accuracy: {accuracy_score(y_test, y_pred):.2f}")


Model Accuracy: 0.30


**Save trained model**

In [25]:
joblib.dump(model, 'chatbot_model.pkl')


['chatbot_model.pkl']

**Load intents**

In [27]:
with open('intents.json', 'r') as f:
    intents = json.load(f)['intents']


**Load transformer model**

In [28]:
try:
    transformer_model = pipeline(
        "text-classification",
        model="distilbert-base-uncased-finetuned-sst-2-english",
        return_all_scores=False
    )
except:
    transformer_model = None
    print("Transformer model could not be loaded. Using fallback classification.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


**Define response function**

In [29]:
def get_response(message):
    processed_input = preprocess_text(message)
    try:
        if transformer_model:
            transformer_result = transformer_model(message[:512])
            if transformer_result[0]['label'] == 'NEGATIVE' and 'not working' in message.lower():
                intent_tag = "software_issue"
            else:
                intent_tag = model.predict([processed_input])[0]
        else:
            intent_tag = model.predict([processed_input])[0]
    except:
        intent_tag = model.predict([processed_input])[0]

    for intent in intents:
        if intent['tag'] == intent_tag:
            return random.choice(intent['responses'])
    return "I'm not sure how to help with that. Could you please provide more details?"


**Gradio chatbot UI**

In [30]:
def chatbot_interface(message, history):
    return get_response(message)

demo = gr.ChatInterface(
    fn=chatbot_interface,
    title="Enterprise Support Chatbot",
    description="Ask me about password resets, system access, software issues, and more!"
)

demo.launch(share=True)


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0094a4f339f6fdd37a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**Save full Python file**

In [31]:
with open('chatbot.py', 'w') as f:
    f.write("# Full chatbot code goes here...")

print("Chatbot code saved to chatbot.py")
print("Model saved to chatbot_model.pkl")
print("Intents saved to intents.json")


Chatbot code saved to chatbot.py
Model saved to chatbot_model.pkl
Intents saved to intents.json
